# <center> 1. Data Cleaning and Preprocessing

In [73]:
# importing all necessary libraries

import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

####  1. Merging the CGM data for all 25 patients into one data frame, adding patient_id column as an identifier and then joining with the demographic and sleep info file on the patient_id column. Having all the data in one data frame will make it easy to create visualizations.


In [75]:
# Step 1: Load all CGM files
path = "../data/raw/HUPA*.csv"
files = glob.glob(path)

df_list = []
for f in files:
    temp = pd.read_csv(f, delimiter=";", header=0)  # delimiter in the files in a semicolon
    temp['patient_id'] = os.path.basename(f).replace(".csv", "")
    df_list.append(temp)

dfCGMData = pd.concat(df_list, ignore_index=True)

# Step 2: Load patient metadata
dfDemographicsData = pd.read_csv("../data/raw/T1DM_patient_sleep_demographics_with_race.csv")

dfDemographicsData.rename(columns={"Patient_ID": "patient_id"}, inplace=True)

# Step 3: Merge on patient_id
dfT1DData = dfCGMData.merge(dfDemographicsData, on="patient_id", how="left")

# Step 4: Check result
dfT1DData.head()


,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,patient_id,Age,Gender,Race,Average Sleep Duration (hrs),Sleep Quality (1-10),% with Sleep Disturbances
0,2018-06-13T18:40:00,332.0,6.3595,82.322835,34.0,0.091667,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
1,2018-06-13T18:45:00,326.0,7.7280,83.740157,0.0,0.091667,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
2,2018-06-13T18:50:00,330.0,4.7495,80.525180,0.0,0.091667,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
3,2018-06-13T18:55:00,324.0,6.3595,89.129032,20.0,0.091667,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
4,2018-06-13T19:00:00,306.0,5.1520,92.495652,0.0,0.075000,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80


#### 2.Checking that each row in CGM Data files found a correspoding patient demography and sleep data row, thus ensuring there are no patients for whom CGM data exists but demography and sleep data doesn't exist.


In [77]:

missing_demographics = dfT1DData['Age'].isna().sum()
print(f"Number of CGM rows without matching demographics: {missing_demographics}")

Number of CGM rows without matching demographics: 0


#### 3. Checking if any of the columns have null values.

In [79]:
print(dfT1DData['patient_id'].nunique(), "unique patients in CGM data")
print(dfDemographicsData['patient_id'].nunique(), "unique patients in metadata")

print(dfT1DData['patient_id'].unique())

dfT1DData.isna().sum()

25 unique patients in CGM data
25 unique patients in metadata
['HUPA0001P' 'HUPA0002P' 'HUPA0003P' 'HUPA0004P' 'HUPA0005P' 'HUPA0006P'
 'HUPA0007P' 'HUPA0009P' 'HUPA0010P' 'HUPA0011P' 'HUPA0014P' 'HUPA0015P'
 'HUPA0016P' 'HUPA0017P' 'HUPA0018P' 'HUPA0019P' 'HUPA0020P' 'HUPA0021P'
 'HUPA0022P' 'HUPA0023P' 'HUPA0024P' 'HUPA0025P' 'HUPA0026P' 'HUPA0027P'
 'HUPA0028P']


time                            0
glucose                         0
calories                        0
heart_rate                      0
steps                           0
basal_rate                      0
bolus_volume_delivered          0
carb_input                      0
patient_id                      0
Age                             0
Gender                          0
Race                            0
Average Sleep Duration (hrs)    0
Sleep Quality (1-10)            0
% with Sleep Disturbances       0
dtype: int64

#### 4. Rounding all the values in float type columns to 2 decimal places.

In [81]:
float_cols = dfT1DData.select_dtypes(include=['float', 'float64']).columns
print(dfT1DData[float_cols])

dfT1DData[float_cols] = dfT1DData[float_cols].round(2)

dfT1DData.head()

           glucose  calories  heart_rate  steps  basal_rate  \
0       332.000000   6.35950   82.322835   34.0    0.091667   
1       326.000000   7.72800   83.740157    0.0    0.091667   
2       330.000000   4.74950   80.525180    0.0    0.091667   
3       324.000000   6.35950   89.129032   20.0    0.091667   
4       306.000000   5.15200   92.495652    0.0    0.075000   
...            ...       ...         ...    ...         ...   
309387  109.333333  10.79280  104.171171    0.0    0.000000   
309388  114.000000   9.80346  103.442623    0.0    0.000000   
309389  118.666667   5.66622   95.542857    0.0    0.000000   
309390  123.333333   5.57628   91.381356    0.0    0.000000   
309391  128.000000   5.57628   99.257812    0.0    0.000000   

        bolus_volume_delivered  carb_input  Average Sleep Duration (hrs)  \
0                          0.0         0.0                           6.3   
1                          0.0         0.0                           6.3   
2              

,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,patient_id,Age,Gender,Race,Average Sleep Duration (hrs),Sleep Quality (1-10),% with Sleep Disturbances
0,2018-06-13T18:40:00,332.0,6.36,82.32,34.0,0.09,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
1,2018-06-13T18:45:00,326.0,7.73,83.74,0.0,0.09,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
2,2018-06-13T18:50:00,330.0,4.75,80.53,0.0,0.09,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
3,2018-06-13T18:55:00,324.0,6.36,89.13,20.0,0.09,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80
4,2018-06-13T19:00:00,306.0,5.15,92.50,0.0,0.08,0.0,0.0,HUPA0001P,34,Male,Other,6.3,4.5,80


#### 5. Summarizing the makeup of columns to spot outliers or incorrect entries.

In [129]:
num_cols = dfT1DData.select_dtypes(include=['float64','int64']).columns
print(num_cols)
print(dfT1DData[num_cols].describe())
print(dfT1DData[num_cols].info())

Index(['glucose', 'calories', 'heart_rate', 'steps', 'basal_rate',
       'bolus_volume_delivered', 'carb_input', 'Age',
       'Average Sleep Duration (hrs)', 'Sleep Quality (1-10)',
       '% with Sleep Disturbances', 'day_number'],
      dtype='object')
             glucose       calories     heart_rate          steps  \
count  309392.000000  309392.000000  309392.000000  309392.000000   
mean      141.425051       8.813100      76.990004      30.825005   
std        57.085587       6.930581      15.546697      84.981109   
min        40.000000       0.000000      32.410000       0.000000   
25%        99.670000       5.850000      64.930000       0.000000   
50%       132.000000       6.280000      75.420000       0.000000   
75%       173.000000       9.190000      85.610000      11.000000   
max       444.000000     106.350000     195.620000     842.000000   

          basal_rate  bolus_volume_delivered     carb_input            Age  \
count  309392.000000           309392.00000

#### 6. The minimum value in bolus_volume_delivered column is -3. Checking the column to find all negative values, since bolus volume of insulin delivered cannot be negative (insulin cannot be removed from the body).

In [109]:
# Step 1: Check how many negative bolus entries exist
neg_bolus = dfT1DData[dfT1DData["bolus_volume_delivered"] < 0]
print(f"Total negative bolus entries: {len(neg_bolus)}")

# Step 2: Check how many patients are affected
patients_with_neg = neg_bolus["patient_id"].nunique()
print(f"Patients with negative bolus values: {patients_with_neg}")

# Step 3: Count negative bolus entries per patient
neg_per_patient = neg_bolus.groupby("patient_id")["bolus_volume_delivered"].count().sort_values(ascending=False)
print("\nNegative bolus entries per patient:")
print(neg_per_patient)


Total negative bolus entries: 4
Patients with negative bolus values: 1

Negative bolus entries per patient:
patient_id
HUPA0017P    4
Name: bolus_volume_delivered, dtype: int64


### 🩺 Data Cleaning Note – Negative Bolus Values

In the **`bolus_volume_delivered`** column, a few records contained **negative values**.  
Based on our research, clinically, negative insulin delivery does not make sense — pumps deliver insulin in **positive units only**.  
Assuming the possible causes could be:  
- Device logging errors,  
- Interrupted or cancelled bolus deliveries,  
- Accidental removal of the pump/patch at the time of bolus delivery,  
- Data artifacts such as carry-over from previous pump actions.  

Since the exact mechanism could not be confirmed and negative insulin delivery is not physiologically meaningful,  
we replaced these negative values with **0**, treating them as *“no bolus delivered”* at that timestamp.  

This preserves data integrity while avoiding misleading calculations of insulin usage.

#### 7. Since there are only 4 negative values and all of them belong to one patient (HUPA0017P), we can assume they are data entry errors. We will set these values to 0.

In [115]:
dfT1DData["bolus_volume_delivered"] = dfT1DData["bolus_volume_delivered"].clip(lower=0)
print("Any negatives left?", (dfT1DData["bolus_volume_delivered"] < 0).any())

Any negatives left? False


#### 8. Time column currently contains a T in the values before the time element. We need to convert this column into a datetime data type.

In [97]:
dfT1DData['time'].describe()

count                  309392
unique                 228802
top       2019-07-11T06:50:00
freq                        4
Name: time, dtype: object

In [99]:
dfT1DData["time"] = pd.to_datetime(dfT1DData["time"], errors="coerce")

# Check result
print(dfT1DData["time"].dtype)
print(dfT1DData["time"].head())

datetime64[ns]
0   2018-06-13 18:40:00
1   2018-06-13 18:45:00
2   2018-06-13 18:50:00
3   2018-06-13 18:55:00
4   2018-06-13 19:00:00
Name: time, dtype: datetime64[ns]


#### 9. Creating a day_label column to capture the day number of the 14 day study. This column will be useful while creating visualizations.

In [120]:
dfT1DData["date"] = dfT1DData["time"].dt.date

# For each patient, get the unique sorted dates
dfT1DData["day_number"] = (
    dfT1DData.groupby("patient_id")["date"]
    .transform(lambda x: (pd.factorize(x.sort_values().unique())[0] + 1)[pd.Series(x).rank(method="dense").astype(int) - 1])
)

# Create labels like "Day 1", "Day 2", etc.
dfT1DData["day_label"] = "Day " + dfT1DData["day_number"].astype(str)

In [124]:
# Checking results
print(dfT1DData[dfT1DData["patient_id"] == "HUPA0001P"][["time", "date", "day_label"]].head(30))

                  time        date day_label
0  2018-06-13 18:40:00  2018-06-13     Day 1
1  2018-06-13 18:45:00  2018-06-13     Day 1
2  2018-06-13 18:50:00  2018-06-13     Day 1
3  2018-06-13 18:55:00  2018-06-13     Day 1
4  2018-06-13 19:00:00  2018-06-13     Day 1
5  2018-06-13 19:05:00  2018-06-13     Day 1
6  2018-06-13 19:10:00  2018-06-13     Day 1
7  2018-06-13 19:15:00  2018-06-13     Day 1
8  2018-06-13 19:20:00  2018-06-13     Day 1
9  2018-06-13 19:25:00  2018-06-13     Day 1
10 2018-06-13 19:30:00  2018-06-13     Day 1
11 2018-06-13 19:35:00  2018-06-13     Day 1
12 2018-06-13 19:40:00  2018-06-13     Day 1
13 2018-06-13 19:45:00  2018-06-13     Day 1
14 2018-06-13 19:50:00  2018-06-13     Day 1
15 2018-06-13 19:55:00  2018-06-13     Day 1
16 2018-06-13 20:00:00  2018-06-13     Day 1
17 2018-06-13 20:05:00  2018-06-13     Day 1
18 2018-06-13 20:10:00  2018-06-13     Day 1
19 2018-06-13 20:15:00  2018-06-13     Day 1
20 2018-06-13 20:20:00  2018-06-13     Day 1
21 2018-06

#### 10. Renaming the 'time' column to 'date_time' and creating two new columns- one to hold the date values only and another to hold the time values only.

In [127]:
dfT1DData.rename(columns={"time": "date_time"}, inplace=True)

# Create separate 'date' and 'time' columns
dfT1DData["date"] = dfT1DData["date_time"].dt.date
dfT1DData["time"] = dfT1DData["date_time"].dt.time

# Check the result
print(dfT1DData[["date_time", "date", "time"]].head())

            date_time        date      time
0 2018-06-13 18:40:00  2018-06-13  18:40:00
1 2018-06-13 18:45:00  2018-06-13  18:45:00
2 2018-06-13 18:50:00  2018-06-13  18:50:00
3 2018-06-13 18:55:00  2018-06-13  18:55:00
4 2018-06-13 19:00:00  2018-06-13  19:00:00
